# 04. Model Evaluation & Analysis

## 🎯 Objectives
This notebook focuses on evaluating the trained Hybrid Quantum Neural Network (HQNN) model. We will:
1.  **Load the Trained Model** from the saved checkpoint.
2.  **Load Test Data** using the preprocessing pipeline.
3.  **Calculate Metrics** (Precision, Recall, F1-Score, AUC).
4.  **Visualize Results** (Confusion Matrix, ROC Curve).

In [ ]:
import sys
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# Ensure we can import from the ml_engine directory
sys.path.append(os.path.abspath('..'))

from ml_engine.dataset import CreditCardDataset
from ml_engine.models.hybrid_nn import HybridQNN
from ml_engine.config import HQNN_PARAMS, QUANTUM_MODEL_PATH

# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

## 🛠️ Step 1: Load Data and Model

In [ ]:
print("🔄 Loading Test Data...")
dataset = CreditCardDataset()
# We only need the test loader here
_, test_loader = dataset.get_torch_dataloaders(batch_size=HQNN_PARAMS['batch_size'])

# Get n_features to initialize model structure
X_sample, _ = next(iter(test_loader))
n_features = X_sample.shape[1]

print(f"✅ Data Loaded! Features: {n_features}")

print(f"🔄 Loading Model from {QUANTUM_MODEL_PATH}...")
model = HybridQNN(n_features, HQNN_PARAMS['n_qubits'], HQNN_PARAMS['n_layers'])

# Load state dict if file exists
if os.path.exists(QUANTUM_MODEL_PATH):
    model.load_state_dict(torch.load(QUANTUM_MODEL_PATH))
    model.eval() # Set to evaluation mode
    print("✅ Model Loaded Successfully!")
else:
    print("❌ Model file not found! Please run training first.")

## 📊 Step 2: Generate Predictions

In [ ]:
y_true = []
y_pred_probs = []

print("🔮 Making Predictions...")
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        y_pred_probs.extend(outputs.numpy().flatten())
        y_true.extend(y_batch.numpy().flatten())

y_true = np.array(y_true)
y_pred_probs = np.array(y_pred_probs)
y_pred_labels = (y_pred_probs > 0.5).astype(int)

print("✅ Predictions Complete!")

## 📈 Step 3: Performance Metrics

In [ ]:
print("Model Classification Report:")
print(classification_report(y_true, y_pred_labels))

### Confusion Matrix

In [ ]:
cm = confusion_matrix(y_true, y_pred_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Legit', 'Fraud'], 
            yticklabels=['Legit', 'Fraud'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix: Hybrid Quantum Model')
plt.show()

### ROC Curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()